In [1]:
import re
import pymorphy2

from pymystem3 import Mystem
from string import punctuation
from stop_words import get_stop_words

# Домашнее задание № 1

## 1. Задание на удаление тегов (2 балла)

Часто в данных, с которыми нам нужно работать помимо текста присутствует ещё какая-то лишняя информация: тэги, ссылки, код, разметка. Она, конечно, не всегда лишняя, но обычно от неё лучше избавиться.

Возьмем в качестве примера статью с Хабрахабра. Она скачана автоматически и в там остались некоторые тэги.

In [18]:
text = '''
Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — <a href="http://www.crazyegg.com" title="Сумасшедшие яйца">CrazyEgg</a>. Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо. Запоминается? Отлично!<br><br><img src="http://img172.imageshack.us/img172/8434/18274658kc4.png" alt="Сумасшедшее яйцо"><br><br><blockquote><h3>Что это такое?</h3><br>
Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.<br>
Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).<br><br><img src="https://habrastorage.org/getpro/habr/post_images/7c7/298/33c/7c729833cd942cc493e68833e3e0f12d.jpg" alt="Тепловое отслеживание популярности"><br></blockquote><br><br><a name="habracut"></a><br><br><blockquote><h3>Для кого это?</h3><br>
Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуитивный интерфейс, и хороший дизайн дает о себе знать — сервис массово популяризировался пол года-год назад. Относительно недорогой, но уж точно не из дешевых — сервис предполагает собой 4 платных линии и одну бесплатную. Дабы написать этот обзор я не поленился заплатить 19 долларов (в месяц) выбрав средний вариант — для нескольких проектов с включенными дополнительными функциями.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/0b0/0a0/b16/0b00a0b16b1eda28e35f39487dcd2545.jpg" alt="Отслеживание ссылок"></blockquote><br><blockquote><img src="https://habrastorage.org/getpro/habr/post_images/0b5/433/892/0b54338921665ffb5a90930147296f5b.jpg" alt="Список по популярности"><br><br>
Да-да, сервис не бесплатен. Точнее бесплатная возможность потестировать есть, но она немного обрезана (можно отслеживать только 5000 посетителей и всего 4 страницы на сайте (внимание — 4 страницы, а не сайтов), т.е. вполне хватает для того, чтобы понять полезность сервиса). Зачем это?</h3><br>
С помощью этого сервиса можно тасовать блоки на сайте, которые полезны пользователям больше всего. Больше не нужно спорить создателям — какой блок где расположить. В этом им поможет <a href="http://ttp://www.crazyegg.com" title="Сумасшедшие яйца">Сумасшедшее яйцо</a>.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/38c/af8/e75/38caf8e753782a01dc6419d3902edd57.jpg" alt="Добавление проекта"></blockquote><br><blockquote>Также этот сервис поможет вам понять — в какой зоне сайта лучше всего располагать рекламу, когда в вашем сервисе речь зайдет о монетизации. Ведь альтруизм это хорошо, а деньги на содержание сервиса нужны, и не лишним будет вычислить зоны где реклама будет приносить наибольшую отдачу, и наименьшее раздражение у пользователей.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/0dd/3fb/b34/0dd3fbb34709e77f3fbcd6523c7eac77.jpg" alt="еще полезности"></blockquote><br><blockquote><h3>Как это работает?</h3><br>
Никаких километровых скриптов вставлять не нужно, достаточно вставиь 2 строчки яваскрипта, и сервис начнет отслеживание. Насколько я понял — исполнительный скрипт работает на сервере <a href="http://www.crazyegg.com" title="Сумасшедшие яйца">CrazyEgg</a>, поэтому ваш сайт от этого в производительности не потеряет ни секунды, а полезность довольно таки большая.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/a90/e4e/e67/a90e4ee6760978463e6306a2f5982e24.jpg" alt="управление популярностью"></blockquote><br><blockquote><h3>Сколько это стоит?</h3><br>
Как я уже упоминал — сервис далеко не бесплатен, хоть и имеет тестовую-бесплатную версию. Расскажу подробнее о тарификации.</blockquote><br><blockquote>1. Бесплатная тестовая версия. Включает в себя возможность отслеживания 5 000 посещений, 4 страницы на сайте.<br><br>
2. Базовая версия. В этой версии можно отследить 10 000 посещений, и 10 страниц, что вполне достаточно для среднего корпоративного сайта. Стоит базовый комплект — 9 долларов в месяц. В этот комплект включены все дополнительные функции.<br><br>
3. Версия «Стандарт». В неё входит возможность отслеживания 25 000 посещений на 20 страницах. Вполне подходит для тестирования нового стартапа. Стоит она 19 долларов в месяц, именно её я купил для тестирования сервиса, и написания этого обзора.<br><br>
4. Версия «Плюс». Отличается от предыдущей возможностью отслеживания 100 000 посещений, 50 страниц. Очень хороший тариф для крупных сервисов. Стоит 49 долларов в месяц. Довольно большие деньги за сервис, но они обычно с лихвой окупаются.<br><br>
5. Версия «Про». Стоит почти 100 долларов, имеет возможность отследить 250 000 посещений на ста страницах. Тариф подходит для монстров с большой посещаемостью и большим количеством страниц.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/f73/fb2/62d/f73fb262da4618a8dde67690cfd191ea.jpg" alt="Отслеживание статистики"></blockquote><br><blockquote><h3>Есть и аналоги</h3><br>
Я не поленился, и собрал еще пару ссылок с аналогами, которые предлагают такие же услуги, но немного дешевле.</blockquote><br><blockquote> 1. <a href="https://www.google.com/analytics/home/?hl=en" title="Шикарный сбор и анализ статистики">Google Analytics</a> — бесплатный сервис для сбора и анализа статистики, вывод статистики в наиболее наглядной форме, и без разнообразных рейтингов, счетчиков. Очень подробная и полезная вещь. Рекомендую, т.к. сам пользуюсь им для этого блога.<br><br>
2. <a href="http://www.mapsurface.com/" title="Сервис для отслеживания популярности блоков на сайте">MapSurface</a> — сам еще не использовал (т.к. предпочел CrazyEgg), но врядли будучи плохим сервис собрал бы множество положительных отзывов. К сожалению он сейчас находится в статусе закрытой беты.</blockquote><br><blockquote><h3>Вывод</h3><br>
Использовать можно, и нужно. Вот только тарифы довольно больно кусаются, но обычно эти деньги потом с лихвой отбиваются на повышении конвертации посетителей в деньги. Использовать сервис нужно для тестирования рекламных мест и удобства отдельных страниц, что помогает опять же повысить конвертацию. В общем и целом — <a href="http://ttp://www.crazyegg.com">полезная вещь</a> для каждого владельца сайтов, а для юзабилиста вообще практически обязательна. К счастью для людей, которые поиздержались деньгами в этом месяце — есть полезные аналоги.<br><br><img src="http://img241.imageshack.us/img241/2346/confettitnml5.jpg" alt="Интересная идея визуализации популярности - конфети"></blockquote><br><br>
Автор: <a href="http://www.birzool.com/" title="Я пишу о юзабилити веб интерфейсов">Ярослав Бирзул</a> (DezmASter).<br>
Источник: <a href="http://www.birzool.com/crazyegg/" title="Сумасшедшие яйца, первоисточник">Блог о юзабилити веб интерфейсов</a>.<br><br>
PS: Всех с прошедшим Новым годом! От всей души желаю вам всего самого-самого лучшего, чего вы желаете только в самых сокровенных мечтах. Удачно вам провести время.
'''

In [47]:
def remove_tags(text):
    text = re.sub(r'<.*?>', ' ', text)
    text = re.sub(r'(\w+) ([,.!?:;])', r'\1\2', text)
    text = re.sub('\s+', ' ', text)
    return text.strip()

In [49]:
new_text = remove_tags(text)

In [184]:
print(new_text)

Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg. Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо. Запоминается? Отлично! Что это такое? Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов. Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые). Для кого это? Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуитивный интерфейс, и хороший дизайн д

С помощью регулярных выражение очистите текст от тэгов (мета информацию внутри тэгов тоже удалите). В получившемся тексте точки не должны слипаться с началом следующего предложений, также не должно быть лишних пробелов.

## 2. Задание на выбор слов после "не " (1 балл)

Вытащите из очищенного текста все слова, которые идут после частицы "не" (случаи, когда "не" - часть предыдущего слова, не должны извлекаться), **кроме слов, начинающихся на гласную букву**. Сама часть "не " не должна извлекаться.

In [190]:
re.findall(r'\bне ([цкнгшщзхфвпрлджчсмтбй]{1}\w+)', new_text)

['знаю',
 'поленился',
 'бесплатен',
 'сайтов',
 'нужно',
 'лишним',
 'нужно',
 'потеряет',
 'бесплатен',
 'поленился']

## 3. Задание на выбор глаголов (1 балл)

Попробуйте вытащить из очищенного текста как можно больше глаголов (минимум 40 с повоторениями), не перечисляя их в паттерне полностью. Можете токенизировать текст и использовать re.match, а можете использовать re.findall с \\b в начале и конце паттерна, чтобы не матчить части слов.

В результате не должно быть не глаголов. Также постарайтесь сделать паттерн максимально коротким.

In [191]:
len(re.findall(r'\w+ете|\w+ать|\w+ает|\w+ился|\w+дит|\w+тся|\w+ить|\w+ал\b', new_text))

46

In [192]:
re.findall(r'\w+ете|\w+ать|\w+ает|\w+ился|\w+дит|\w+тся|\w+ить|\w+ал\b', new_text)

['работает',
 'Запоминает',
 'отслеживать',
 'выводит',
 'совмещает',
 'Разумеется',
 'пользовать',
 'дает',
 'знать',
 'предполагает',
 'написать',
 'поленился',
 'заплатить',
 'потестировать',
 'отслеживать',
 'хватает',
 'тасовать',
 'спорить',
 'расположить',
 'располагать',
 'вычислить',
 'приносить',
 'работает',
 'работает',
 'производит',
 'упоминал',
 'Включает',
 'отследит',
 'входит',
 'подходит',
 'Отличает',
 'окупаются',
 'отследит',
 'подходит',
 'поленился',
 'собрал',
 'использовал',
 'собрал',
 'находит',
 'Использовать',
 'кусаются',
 'отбиваются',
 'Использовать',
 'помогает',
 'повысить',
 'желаете']

## 4. Pymorphy vs Mystem (2 балла)

Лемматизируйте очищенный текст с помощью mystem и pymorphy. Сравните результаты: есть ли разница, что лучше. 

Для пайморфи используйте токенизацию из mystem, чтобы исключить влияние токенизации на результат.

In [201]:
russian_stopwords = get_stop_words('ru')

mystem = Mystem(disambiguation=False) 
punc = punctuation
punc += '—«»'

def preprocess_text_mystem(text):
    tokens = []
    for token in mystem.lemmatize(text):
        token = token.strip()
        if token:
            if token not in punc and token[0] not in punc and token not in russian_stopwords:
                tokens.append(token)
    return " ".join(tokens)

In [153]:
morph = pymorphy2.MorphAnalyzer()

def preprocess_text_morph(text):
    tokens = []

    for token_info in mystem.analyze(text):
        token = token_info['text'].strip()
        if token:
            if token not in punc and token[0] not in punc and token not in russian_stopwords:
                tokens.append(morph.parse(token)[0].normal_form)
    return ' '.join(tokens)

In [203]:
preprocess_text_mystem(new_text)

'сегодняшний заметка сервис отслеживание активность пользователь CrazyEgg знать обязанный сервис говорить работать запоминаться отлично высоко сервис отслеживание перемещение пользователь сайт кликнуть ссылка популярный подобный разнородный информация юзабилист сервис позволять отслеживать активность определенный пользователь выводить данные различный форма инфракрасный активно область теплый салют активно область конфеть простой список сортировка активность колба заполненный колба активно область облако взгляд удобный вариант совмещать остальной вместе взять разуметься сервис изначально планироваться помощник юзабилист пользоваться деньги интуитивный интерфейс хороший дизайн давать знать сервис массово популяризироваться пол относительно недорогой точно дешевый сервис предполагать 4 платный линия бесплатный дабы написать обзор полениться заплатить 19 доллар месяц выбирать средний вариант проект включать дополнительный функция сервис бесплатный точно бесплатный возможность потестироват

In [204]:
preprocess_text_morph(new_text)

'сегодняшний заметка сервис отслеживание активность пользователь crazyegg я знать обязать сервис такой говорящий имя работать запоминаться отлично что как сказать выше сервис отслеживание перемещение пользователь сайт кликнуть какой ссылка популярный подобный разнородный информация юзабилист сервис позволять отслеживать активность определённый пользователь выводить дать различный форма инфракрасный активный область тёплый салют активный область конфести простой список сортировка активность колба залненный колба активный область облако взгляд удобный вариант совмещать остальной вместе взять для разумеется сервис изначально планироваться помощник юзабилист пользоваться деньга интуитивный интерфейс хороший дизайн давать знать сервис массово популяризироваться пол относительно недорогой точно дешёвый сервис предполагать 4 платный линия один бесплатный дабы написать обзор полениться заплатить 19 доллар месяц выбрать средний вариант несколько проект включить дополнительный функция да сервис 

In [202]:
len(list(set(preprocess_text_mystem(new_text).split(' ')))), len(list(set(preprocess_text_morph(new_text).split(' '))))

(249, 286)

Разница в том, что у Pymorphy начальная форма может быть указана от различных частей речи, мой код берет нулевой элемент в этом списке, т.е. тот у которого наибольший скор по мнению пайморфи. В то время как mystem выбирает единственный вариант. Сказать, что лучше - сложно, но мне кажется что разница будет минимальной. Если смотреть на размер получившегося словаря, то у майстема он меньше на 37 слов, чем у пайморфи. Т.к. леммы одинаковые, то искусственное раздутие словаря за счет различных словоформ смысла не имеет, поэтому предпочтение отдаю mystem.

## 5. Разбиение на предложения в Mystem (2 балла)

С помощью Mystem разделите текст из первого задания (очищенный) на предложения. Сравните разбиение с razdel.sentenize

In [160]:
with open('text.txt', 'w', encoding='utf-8') as f:
    f.write(new_text)

In [161]:
!mystem -isc --format json text.txt text_parsed.txt

In [163]:
tokens = []

with open('text_parsed.txt', 'r', encoding='utf-8') as f:
    for row in f:
        tokens.append(row)

In [172]:
[sentence.strip() for sentence in ''.join([token['text'] for token in eval(tokens[0])]).split('\\s')]

['Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg.',
 'Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо.',
 'Запоминается?',
 'Отлично!',
 'Что это такое?',
 'Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.',
 'Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).',
 'Для кого это?',
 'Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуити

In [174]:
from razdel import sentenize

In [182]:
[sentence.text for sentence in sentenize(new_text)]

['Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg.',
 'Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо.',
 'Запоминается?',
 'Отлично!',
 'Что это такое?',
 'Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.',
 'Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).',
 'Для кого это?',
 'Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуити

<b>Razdel справился лучше, отделил корректно нумерацию списка, в остальном работает также.</b>

## 6*. Дизамбигуация в Mystem (2 балла)

*необязательное задание на 10 баллов

Одна из фичей майстема - контекстная дизамбигуация (т.е. определение правильного разбора в зависимости от контекста).

Запустите mystem на очищенном тексте с дизамбигуацией и без и проверьте есть ли какие-то различия.

In [205]:
mystem = Mystem(disambiguation=True) 

new_text_dis = preprocess_text_mystem(new_text)

mystem = Mystem(disambiguation=False) 

new_text_undis = preprocess_text_mystem(new_text)

In [213]:
new_text_dis

'сегодняшний заметка сервис отслеживание активность пользователь CrazyEgg знать обязанный сервис говорящий работать запоминаться отлично выше сервис отслеживание перемещение пользователь сайт кликнуть ссылка популярный подобный разнородный информация юзабилист сервис позволять отслеживать активность определенный пользователь выводить данные различный форма инфракрасный активно область тепло салют активно область конфеть простой список сортировка активность колба заполненный колба активно область облако взгляд удобный вариант совмещать остальной вместе взять разумеется сервис изначально планироваться помощник юзабилист пользоваться деньги интуитивный интерфейс хороший дизайн давать знать сервис массово популяризироваться пол относительно недорогой точно дешевый сервис предполагать 4 платный линия бесплатный дабы написать обзор полениться заплатить 19 доллар месяц выбирать средний вариант проект включать дополнительный функция сервис бесплатный точно бесплатный возможность потестировать 

In [212]:
new_text_undis

'сегодняшний заметка сервис отслеживание активность пользователь CrazyEgg знать обязанный сервис говорить работать запоминаться отлично высоко сервис отслеживание перемещение пользователь сайт кликнуть ссылка популярный подобный разнородный информация юзабилист сервис позволять отслеживать активность определенный пользователь выводить данные различный форма инфракрасный активно область теплый салют активно область конфеть простой список сортировка активность колба заполненный колба активно область облако взгляд удобный вариант совмещать остальной вместе взять разуметься сервис изначально планироваться помощник юзабилист пользоваться деньги интуитивный интерфейс хороший дизайн давать знать сервис массово популяризироваться пол относительно недорогой точно дешевый сервис предполагать 4 платный линия бесплатный дабы написать обзор полениться заплатить 19 доллар месяц выбирать средний вариант проект включать дополнительный функция сервис бесплатный точно бесплатный возможность потестироват

Различия есть, с дизамбигуацией учитывается контекст, в котором анализируемое слово находится, поэтому как по мне, то с ней лемматизация работает лучше. Например, слово "говорящим" приведено к лемме "говорящий" с дизамбигуацией и "говорить" без нее - в задаче классификации, скорее данное различие никак не отразится, но например построение векторов на этих леммах с помощью word2vec (в которых как раз учитывается контекст) возможно даст более качественный результат.